<a href="https://colab.research.google.com/github/yanglinjing/dand_p7_data_wragling/blob/master/wrangle_act2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wrangle WeRateDogs Twitter Data


# Goal
Wrangle **WeRateDogs Twitter** data to create interesting and trustworthy analyses an visualizations. 

In [0]:
import pandas as pd
import requests
import csv
import tweepy
import os
import json
import numpy as np

In [4]:
# Check current working directory

!pwd

/content


In [5]:
# All the documents are stored on Google Drive.
# It requires authorisation before use them.
# This chunk of code is about authorisation.

# Mounting Google Drive locally
# https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
# Change Working Directory in Google Drive

os.chdir('gdrive/My Drive/Colab Notebooks/dand_p7_data_wragling')
!pwd

# Entire Url: '/content/gdrive/My Drive/Colab Notebooks/dand_p7_data_wragling'

/content/gdrive/My Drive/Colab Notebooks/dand_p7_data_wragling


In [0]:
# Create a new folder if it does not exist

folder_name = 'raw_data'
if not os.path.exists(folder_name):
  os.makedirs(folder_name)

# Data Gathering

## Enhanced Twitter Archive
The original data contains basic tweet data for all 5000+ of **WeRateDogs'** tweets, including rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo).

The current dataset is filtered for tweets with ratings only (there are **2356**).
 
However, some of the ratings,  dog names and dog stages probably aren't all correct.
 
Besides, some of the rating numerators are greater than the denominators - this is a unique rating system of WeRateDogs.

No data has been gathered beyond August 1st, 2017.

In [0]:
### 1st df: WeRateDogs Twitter archive

df_1_path = os.path.join(folder_name, 'twitter-archive-enhanced.csv')

with open(df_1_path, 'r') as f:
  df_1 = pd.read_csv(f)

In [9]:
df_1.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In [10]:
df_1.shape

(2356, 17)

## Additional Data via the Twitter API
**Retweet count** and **favorite count** are two of the notable column omissions of Twitter archives.

Fortunately, this additional data can be gathered from Twitter's API. 

In [0]:
### 2nd df: 
# Each tweet's retweet count and favorite ("like") count

# Using the tweet IDs in the WeRateDogs Twitter archive, 
# query the Twitter API for each tweet's JSON data 
# using Python's Tweepy library:
# http://www.tweepy.org/

consumer_key = 'pTzzUxLTJcdNCeAHOvhUbflHX'
consumer_secret = 'bW1RvcK3vlFBxYFe9XozytZTR50hVsZmdpRCBV2zKFKUSBYEOG'
access_token = '1090182192024629249-EMhWG3aaCj1QPeMd8bHO21XPYDRQse'
access_secret = '0qWGdlA4iCQ7wFw0RK1FXC6XKHfXtzYryvil0wcyzmN3j'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True)
# wait_on_rate_limit = True
# Note: do not include your Twitter API keys, secrets, and tokens

In [0]:
# test

# get status object
status = api.get_status('892420643555336193') 

# get json strings from status obj
json_str = json.loads(json.dumps(status._json))

# get retweet/like counts from json strings
retweet_count = json_str['retweet_count']

In [90]:
print(retweet_count)

8302


In [122]:
df_2_list = []

# Use each id in df_1
for id in df_1['tweet_id']:
  

  # get status object
  status = api.get_status(id) 

  # get json strings from status obj
  json_str = json.loads(json.dumps(status._json))

  # get retweet/like counts from json strings
  retweet_count = json_str['retweet_count']
  favorite_count = json_str['favorite_count']

  # write to list
  df_2_list.append(f"{id} {retweet_count} {favorite_count}")


  break

df_2_list

['892420643555336193 8302 37994']

In [0]:
# Use id provided in df_1
# to retrive its retweet & favorite count, and
# temporarily store in a list

df_2_list = []

# Use each id in df_1
for id in df_1['tweet_id']:
  
  try:
    # get status object
    status = api.get_status(id) 

    # get json strings from status obj
    json_str = json.loads(json.dumps(status._json))

    # get retweet/like counts from json strings
    retweet_count = json_str['retweet_count']
    favorite_count = json_str['favorite_count']
    
    # write to list
    df_2_list.append(f"{id} {retweet_count} {favorite_count}")

  # in case there is no status obj returned
  except Exception as e:
    df_2_list.append(f"{id} Nan Nan")


In [0]:
# store each tweet's info in a file called tweet_json.txt file. 
# Each tweet's JSON data should be written to its own line. 

df_2_path = os.path.join(folder_name, 'tweet_json.txt')

with open(df_2_path, "w") as f:
  for item in df_2_list:
    f.write(item + '\n')
    

In [0]:
# Read this .txt file line by line into a pandas DataFrame 
# with tweet ID, retweet count, and favorite count. 

df_2 = pd.read_csv(df_2_path,  sep=" ", 
                  names = ['id', 'retweet_count', 'favorite_count'])


In [144]:
df_2.head()

,id,retweet_count,favorite_count
0,892420643555336193,8302,37994
1,892177421306343426,6131,32617
2,891815181378084864,4060,24560
3,891689557279858688,8434,41319
4,891327558926688256,9147,39505


In [141]:
df_2.shape

(2356, 3)

## Image Predictions File

Every image in the WeRateDogs Twitter archive was through a neural network that can classify breeds of dogs.

(The neural network: https://www.youtube.com/watch?v=2-Ol7ZB0MmU)

The results: a table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the **most confident prediction** (**numbered 1 to 4** since tweets can have up to four images).

In the table,
- **tweet_id** is the last part of the tweet URL after "status/" → https://twitter.com/dog_rates/status/889531135344209921
- **p1** is the algorithm's **#1 prediction** for the image in the tweet → golden retriever
- **p1_conf** is how confident the algorithm is in its #1 prediction → 95%
- **p1_dog** is whether or not the #1 prediction is a breed of dog → TRUE
- **p2** is the algorithm's **2nd most likely prediction** → Labrador retriever
- -etc.

In [23]:
### 3rd df: tweet image predictions

# The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) 
# is present in each tweet according to a neural network. 

# This file (image_predictions.tsv) is hosted on Udacity's servers and 
# should be downloaded programmatically using the Requests library 
# Requests library: http://docs.python-requests.org/en/master/


# Store the content into response in working memory

url_3 = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url_3)

response
# <Response [200]>: success

<Response [200]>

In [0]:
# folder name / file name (shown in url_2):
# 'raw_data/image-predictions.tsv'

df_3_path = os.path.join(folder_name, url_3.split('/')[-1])

# put the content of response into f
with open (df_3_path, mode='wb') as f:
  f.write(response.content)
  
# ensure the new file has been built
# os.listdir(folder_name)

In [25]:
df_3 = pd.read_csv(df_3_path, sep='\t')

df_3.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [26]:
df_3.shape

(2075, 12)

# Data Cleaning


## Quality issues


###  Table 1: Enhanced Twitter Archive

Missing Values:


-  in_reply_to_status_id: 78 out of 2356 is non-null
- in_reply_to_user_id: 78 out of 2356 is non-null
- retweeted_status_id: 181 out of 2356 is non-null
- retweeted_status_user_id:  181 out of 2356 is non-null
- retweeted_status_timestamp:  181 out of 2356 is non-null
- name: string 'None' should be replaced by Null


Data Types:

- tweet_id: int -> obj
-  timestamp: obj -> datetime
-  in_reply_to_status_id: float -> object
- in_reply_to_user_id: float -> object
- retweeted_status_id: float -> object
- retweeted_status_user_id: float -> object

Accuracy:

- source: the long name with HTML tag could be shorten.
- rating_denominator: contains values zero and 2, which might not be an effective value.

In [56]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [117]:
df_1.sample(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
355,830956169170665475,NaN,NaN,2017-02-13 01:46:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Reggie. He hates puns. 12/10 ligh...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/830956169...,12,10,Reggie,None,None,None,None
176,857746408056729600,NaN,NaN,2017-04-28 00:00:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Thor. He doesn't have finals because he's...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/857746408...,13,10,Thor,None,None,None,None
2249,667861340749471744,NaN,NaN,2015-11-21 00:25:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Shotokon Macadamia mix named Cheryl....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667861340...,9,10,a,None,None,None,None


In [64]:
df_1.duplicated().sum()

0

In [70]:
df_1['doggo'].value_counts()

None     2259
doggo      97
Name: doggo, dtype: int64

In [71]:
df_1['floofer'].value_counts()

None       2346
floofer      10
Name: floofer, dtype: int64

In [72]:
df_1['pupper'].value_counts()

None      2099
pupper     257
Name: pupper, dtype: int64

In [73]:
df_1['puppo'].value_counts()

None     2326
puppo      30
Name: puppo, dtype: int64

In [74]:
df_1['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [120]:
df_1['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

### Table 2: Additional Infomation (Retweet & Favourate)

Missing Values:

- retweet & favouriate count: "Nan" should be replaced by Null.
- retweet & favouriate count: 16 missing values

In [0]:
# Replace str 'Nan' with Null

df_2_copy = df_2.replace('Nan', np.NaN)

In [138]:
df_2_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 3 columns):
id             2356 non-null int64
retweet_n      2340 non-null object
favourate_n    2340 non-null object
dtypes: int64(1), object(2)
memory usage: 55.3+ KB


### Table 3: Image Predictions

Data Types:

- tweet_id: int -> obj
- img_num: int -> obj
- p1_dog / p2_dog / p3_dog: obj -> bool

In [65]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [77]:
df_3.sample(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
692,684222868335505415,https://pbs.twimg.com/media/CX7Y_ByWwAEJdUy.jpg,1,soft-coated_wheaten_terrier,0.791182,True,cocker_spaniel,0.072444,True,teddy,0.071486,False
1322,756526248105566208,https://pbs.twimg.com/media/Cn-4m2CXYAErPGe.jpg,1,geyser,0.991273,False,volcano,0.004673,False,fountain,0.001234,False
510,676146341966438401,https://pbs.twimg.com/media/CWIngp5WEAAJOy3.jpg,1,Irish_water_spaniel,0.388332,True,standard_poodle,0.284121,True,greenhouse,0.034868,False


In [145]:
df_3.duplicated().sum()

0

In [78]:
df_3['p1'].value_counts()

golden_retriever             150
Labrador_retriever           100
Pembroke                      89
Chihuahua                     83
pug                           57
chow                          44
Samoyed                       43
toy_poodle                    39
Pomeranian                    38
cocker_spaniel                30
malamute                      30
French_bulldog                26
Chesapeake_Bay_retriever      23
miniature_pinscher            23
seat_belt                     22
Siberian_husky                20
German_shepherd               20
Staffordshire_bullterrier     20
Cardigan                      19
web_site                      19
Eskimo_dog                    18
teddy                         18
beagle                        18
Maltese_dog                   18
Shetland_sheepdog             18
Lakeland_terrier              17
Shih-Tzu                      17
Rottweiler                    17
Italian_greyhound             16
kuvasz                        16
          

## Tidiness Issues 


### Table 1:

- doggo	/ floofer	/ pupper /	puppo: could be combined into 1 column as categorical data.